In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from copy import deepcopy
from datetime import timedelta

import plotly.offline
import cufflinks as cf
import plotly.express as px
from plotly.offline import iplot
import plotly.graph_objects as go
from IPython.core.interactiveshell import InteractiveShell

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
InteractiveShell.ast_node_interactivity = "all"

# Блок 1 Импорт данных

In [2]:
%%time
df = pd.read_csv('/home/brainiac/Vlad/Git Files/CryptoLp DataScience/DataSets/Orderbook/Main_Orderbook.csv')

CPU times: user 48.6 s, sys: 8.35 s, total: 56.9 s
Wall time: 1min 5s


In [3]:
df.head(3)

,datetimes,ask_0_price,ask_1_price,ask_2_price,ask_3_price,ask_4_price,ask_5_price,ask_6_price,ask_7_price,ask_8_price,...,bid_0_size,bid_1_size,bid_2_size,bid_3_size,bid_4_size,bid_5_size,bid_6_size,bid_7_size,bid_8_size,bid_9_size
0,2020-07-29 12:25:06.423343+00:00,11108.5,11109.0,11109.5,11110.0,11110.5,11111.5,11112.0,11112.5,11113.0,...,2470752,183215,27029,13019,123951,124582,129294,46695,104016,41558
1,2020-07-29 12:25:06.488583,11108.5,11109.0,11109.5,11110.0,11110.5,11111.5,11112.0,11112.5,11113.0,...,2470752,183215,27029,13019,123951,124582,129294,46695,104016,41558
2,2020-07-29 12:25:06.489924,11108.5,11109.0,11109.5,11110.0,11110.5,11111.5,11112.0,11112.5,11113.0,...,2470752,183215,27029,13019,123951,124582,129294,46695,104016,41558


In [4]:
df.shape

(25844802, 41)

In [5]:
df['Mid Price'] = (df['ask_0_price']+df['bid_0_price'])/2

## Попытка предсказать скачки цен

In [7]:
df = df[['datetimes','Mid Price',
         'bid_0_size','bid_1_size','bid_2_size','bid_3_size','bid_4_size','bid_5_size','bid_6_size','bid_7_size','bid_8_size','bid_9_size',
         'ask_0_size','ask_1_size','ask_2_size','ask_3_size','ask_4_size','ask_5_size','ask_6_size','ask_7_size','ask_8_size','ask_9_size',
        ]]

In [8]:
df.head(3)

,datetimes,Mid Price,bid_0_size,bid_1_size,bid_2_size,bid_3_size,bid_4_size,bid_5_size,bid_6_size,bid_7_size,...,ask_0_size,ask_1_size,ask_2_size,ask_3_size,ask_4_size,ask_5_size,ask_6_size,ask_7_size,ask_8_size,ask_9_size
0,2020-07-29 12:25:06.423343+00:00,11108.25,2470752,183215,27029,13019,123951,124582,129294,46695,...,61476,22340,129,289,1076,10150,10367,43269,10543,64703
1,2020-07-29 12:25:06.488583,11108.25,2470752,183215,27029,13019,123951,124582,129294,46695,...,61476,22340,129,289,1076,10150,10367,43269,10543,64703
2,2020-07-29 12:25:06.489924,11108.25,2470752,183215,27029,13019,123951,124582,129294,46695,...,61476,22340,129,289,1076,10150,10367,43269,10543,64703


In [9]:
%%time
# Имбалансы первого уровня
df['Imb_1_LvL'] = (df['bid_0_size']-df['ask_0_size'])
df['Imb_1_LvL_W'] = (df['bid_0_size']-df['ask_0_size'])/(df['bid_0_size']+df['ask_0_size'])
df['Log Imb1LvL'] = np.log(df['bid_0_size'])-np.log(df['ask_0_size'])
#***********************

# Имбалансы трех уровней
df['Sum Bid Volume_3'] = df['bid_0_size']+df['bid_1_size']+df['bid_2_size']
df['Sum Ask Volume_3'] = df['ask_0_size']+df['ask_1_size']+df['ask_2_size']

df['Imb_3_LvL'] = (df['Sum Bid Volume_3']-df['Sum Ask Volume_3'])
df['Imb_3_LvL_W'] = (df['Sum Bid Volume_3']-df['Sum Ask Volume_3'])/(df['Sum Bid Volume_3']+df['Sum Ask Volume_3'])
df['Log Imb3LvL'] = np.log(df['Sum Bid Volume_3'])-np.log(df['Sum Ask Volume_3'])
#***********************

# Имбалансы пяти уровней
df['Sum Bid Volume_5'] = df['bid_0_size']+df['bid_1_size']+df['bid_2_size']+df['bid_3_size']+df['bid_4_size']
df['Sum Ask Volume_5'] = df['ask_0_size']+df['ask_1_size']+df['ask_2_size']+df['ask_3_size']+df['ask_4_size']

df['Imb_5_LvL'] = (df['Sum Bid Volume_5']-df['Sum Ask Volume_5'])
df['Imb_5_LvL_W'] = (df['Sum Bid Volume_5']-df['Sum Ask Volume_5'])/(df['Sum Bid Volume_5']+df['Sum Ask Volume_5'])
df['Log Imb5LvL'] = np.log(df['Sum Bid Volume_5'])-np.log(df['Sum Ask Volume_5'])
#***********************

# Имбалансы десяти уровней
df['Sum Bid Volume_10'] = df['bid_0_size']+df['bid_1_size']+df['bid_2_size']+df['bid_3_size']+df['bid_4_size']+\
                          df['bid_5_size']+df['bid_6_size']+df['bid_7_size']+df['bid_8_size']+df['bid_9_size']

df['Sum Ask Volume_10'] = df['ask_0_size']+df['ask_1_size']+df['ask_2_size']+df['ask_3_size']+df['ask_4_size']+\
                          df['ask_5_size']+df['ask_6_size']+df['ask_7_size']+df['ask_8_size']+df['ask_9_size']

df['Imb_10_LvL'] = (df['Sum Bid Volume_10']-df['Sum Ask Volume_10'])
df['Imb_10_LvL_W'] = (df['Sum Bid Volume_10']-df['Sum Ask Volume_10'])/(df['Sum Bid Volume_10']+df['Sum Ask Volume_10'])
df['Log Imb10LvL'] = np.log(df['Sum Bid Volume_10'])-np.log(df['Sum Ask Volume_10'])
#***********************

CPU times: user 13.5 s, sys: 35.3 ms, total: 13.6 s
Wall time: 2.9 s


In [10]:
df.head(3)

,datetimes,Mid Price,bid_0_size,bid_1_size,bid_2_size,bid_3_size,bid_4_size,bid_5_size,bid_6_size,bid_7_size,...,Sum Bid Volume_5,Sum Ask Volume_5,Imb_5_LvL,Imb_5_LvL_W,Log Imb5LvL,Sum Bid Volume_10,Sum Ask Volume_10,Imb_10_LvL,Imb_10_LvL_W,Log Imb10LvL
0,2020-07-29 12:25:06.423343+00:00,11108.25,2470752,183215,27029,13019,123951,124582,129294,46695,...,2817966,85310,2732656,0.941232,3.497479,3264111,224342,3039769,0.87138,2.677571
1,2020-07-29 12:25:06.488583,11108.25,2470752,183215,27029,13019,123951,124582,129294,46695,...,2817966,85310,2732656,0.941232,3.497479,3264111,224342,3039769,0.87138,2.677571
2,2020-07-29 12:25:06.489924,11108.25,2470752,183215,27029,13019,123951,124582,129294,46695,...,2817966,85310,2732656,0.941232,3.497479,3264111,224342,3039769,0.87138,2.677571


In [11]:
df.columns

Index(['datetimes', 'Mid Price', 'bid_0_size', 'bid_1_size', 'bid_2_size',
       'bid_3_size', 'bid_4_size', 'bid_5_size', 'bid_6_size', 'bid_7_size',
       'bid_8_size', 'bid_9_size', 'ask_0_size', 'ask_1_size', 'ask_2_size',
       'ask_3_size', 'ask_4_size', 'ask_5_size', 'ask_6_size', 'ask_7_size',
       'ask_8_size', 'ask_9_size', 'Imb_1_LvL', 'Imb_1_LvL_W', 'Log Imb1LvL',
       'Sum Bid Volume_3', 'Sum Ask Volume_3', 'Imb_3_LvL', 'Imb_3_LvL_W',
       'Log Imb3LvL', 'Sum Bid Volume_5', 'Sum Ask Volume_5', 'Imb_5_LvL',
       'Imb_5_LvL_W', 'Log Imb5LvL', 'Sum Bid Volume_10', 'Sum Ask Volume_10',
       'Imb_10_LvL', 'Imb_10_LvL_W', 'Log Imb10LvL'],
      dtype='object')

In [12]:
df = df[['datetimes','Mid Price',
         'Imb_1_LvL','Imb_1_LvL_W','Log Imb1LvL',
         'Imb_3_LvL','Imb_3_LvL_W','Log Imb3LvL','Sum Bid Volume_3','Sum Ask Volume_3',
         'Imb_5_LvL','Imb_5_LvL_W','Log Imb5LvL','Sum Bid Volume_5','Sum Ask Volume_5',
         'Imb_10_LvL','Imb_10_LvL_W','Log Imb10LvL','Sum Bid Volume_10','Sum Ask Volume_10',
        ]]

In [13]:
df.head(3)

,datetimes,Mid Price,Imb_1_LvL,Imb_1_LvL_W,Log Imb1LvL,Imb_3_LvL,Imb_3_LvL_W,Log Imb3LvL,Sum Bid Volume_3,Sum Ask Volume_3,Imb_5_LvL,Imb_5_LvL_W,Log Imb5LvL,Sum Bid Volume_5,Sum Ask Volume_5,Imb_10_LvL,Imb_10_LvL_W,Log Imb10LvL,Sum Bid Volume_10,Sum Ask Volume_10
0,2020-07-29 12:25:06.423343+00:00,11108.25,2409276,0.951445,3.693631,2597051,0.939279,3.463782,2680996,83945,2732656,0.941232,3.497479,2817966,85310,3039769,0.87138,2.677571,3264111,224342
1,2020-07-29 12:25:06.488583,11108.25,2409276,0.951445,3.693631,2597051,0.939279,3.463782,2680996,83945,2732656,0.941232,3.497479,2817966,85310,3039769,0.87138,2.677571,3264111,224342
2,2020-07-29 12:25:06.489924,11108.25,2409276,0.951445,3.693631,2597051,0.939279,3.463782,2680996,83945,2732656,0.941232,3.497479,2817966,85310,3039769,0.87138,2.677571,3264111,224342


In [14]:
df['Price Change 1'] = df['Mid Price'].diff(periods=1)
df['Price Change 5'] = df['Mid Price'].diff(periods=5)
df['Price Change 10'] = df['Mid Price'].diff(periods=10)
df['Price Change 25'] = df['Mid Price'].diff(periods=25)
df['Price Change 50'] = df['Mid Price'].diff(periods=50)
df['Price Change 100'] = df['Mid Price'].diff(periods=100)
df = df.dropna()
df.index = range(len(df))

In [16]:
df.shape

(25844702, 26)

In [23]:
df[df['Price Change 10']!=0].head()

,datetimes,Mid Price,Imb_1_LvL,Imb_1_LvL_W,Log Imb1LvL,Imb_3_LvL,Imb_3_LvL_W,Log Imb3LvL,Sum Bid Volume_3,Sum Ask Volume_3,...,Imb_10_LvL_W,Log Imb10LvL,Sum Bid Volume_10,Sum Ask Volume_10,Price Change 1,Price Change 5,Price Change 10,Price Change 25,Price Change 50,Price Change 100
250,2020-07-29 12:25:08.637921,11108.50,2543217,0.982880,4.752064,2752402,0.983447,4.786053,2775565,23163,...,0.912208,3.081046,3407164,156427,0.25,0.25,0.25,0.25,0.25,0.25
251,2020-07-29 12:25:08.638116,11108.50,2544518,0.983878,4.812598,2753703,0.984370,4.843860,2775565,21862,...,0.912907,3.089398,3407164,155126,0.00,0.25,0.25,0.25,0.25,0.25
252,2020-07-29 12:25:08.638224,11108.75,44353,0.515439,1.140217,2792175,0.984582,4.857625,2814037,21862,...,0.913647,3.098322,3437707,155126,0.25,0.50,0.50,0.50,0.50,0.50
253,2020-07-29 12:25:08.639295,11108.75,57728,0.580624,1.326808,2805550,0.984654,4.862367,2827412,21862,...,0.913967,3.102205,3451082,155126,0.00,0.50,0.50,0.50,0.50,0.50
254,2020-07-29 12:25:08.639411,11108.75,71805,0.632638,1.491603,2819627,0.984730,4.867334,2841489,21862,...,0.914302,3.106276,3465159,155126,0.00,0.50,0.50,0.50,0.50,0.50


In [29]:
%%time
mid_price = df['Mid Price'].values

Price_changes_percent = list()
Price_changes_percent = [0]
for i in tqdm(range(1, len(df))):
    a = ((mid_price[i]-mid_price[i-1])/mid_price[i-1])*100
    Price_changes_percent.append(a)
df['Pp changes 1'] = Price_changes_percent

Price_changes_percent = list()
Price_changes_percent = [0] * 5
for i in tqdm(range(5, len(df))):
    a = ((mid_price[i]-mid_price[i-5])/mid_price[i-5])*100
    Price_changes_percent.append(a)
df['Pp changes 5'] = Price_changes_percent

Price_changes_percent = list()
Price_changes_percent = [0] * 10
for i in tqdm(range(10, len(df))):
    a = ((mid_price[i]-mid_price[i-10])/mid_price[i-10])*100
    Price_changes_percent.append(a)
df['Pp changes 10'] = Price_changes_percent

Price_changes_percent = list()
Price_changes_percent = [0] * 25
for i in tqdm(range(25, len(df))):
    a = ((mid_price[i]-mid_price[i-25])/mid_price[i-25])*100
    Price_changes_percent.append(a)
df['Pp changes 25'] = Price_changes_percent

Price_changes_percent = list()
Price_changes_percent = [0] * 50
for i in tqdm(range(50, len(df))):
    a = ((mid_price[i]-mid_price[i-50])/mid_price[i-50])*100
    Price_changes_percent.append(a)
df['Pp changes 50'] = Price_changes_percent

Price_changes_percent = list()
Price_changes_percent = [0] * 100
for i in tqdm(range(100, len(df))):
    a = ((mid_price[i]-mid_price[i-100])/mid_price[i-100])*100
    Price_changes_percent.append(a)
df['Pp changes 100'] = Price_changes_percent

100%|██████████| 25844602/25844602 [00:28<00:00, 903776.76it/s]


CPU times: user 3min 11s, sys: 2.49 s, total: 3min 14s
Wall time: 3min 21s


In [30]:
df.head()

,datetimes,Mid Price,Imb_1_LvL,Imb_1_LvL_W,Log Imb1LvL,Imb_3_LvL,Imb_3_LvL_W,Log Imb3LvL,Sum Bid Volume_3,Sum Ask Volume_3,...,Price Change 10,Price Change 25,Price Change 50,Price Change 100,Pp changes 1,Pp changes 5,Pp changes 10,Pp changes 25,Pp changes 50,Pp changes 100
0,2020-07-29 12:25:06.554033,11108.25,2594633,0.980946,4.644061,2782721,0.967003,4.087857,2830198,47477,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-07-29 12:25:06.554647,11108.25,2594633,0.980946,4.644061,2782721,0.967003,4.087857,2830198,47477,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-07-29 12:25:06.555250,11108.25,2594633,0.980946,4.644061,2782721,0.967003,4.087857,2830198,47477,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-07-29 12:25:06.555849,11108.25,2594633,0.980946,4.644061,2782721,0.967003,4.087857,2830198,47477,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-07-29 12:25:06.556386,11108.25,2594633,0.980946,4.644061,2782721,0.967003,4.087857,2830198,47477,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
price_chan = df['Pp changes 1'].values
l_mark = list()
for i in tqdm(range(len(df))):
    if price_chan[i] >= 0.01:
        l_mark.append(1)
    else:
        l_mark.append(0)
df['Markers 1'] = l_mark

price_chan = df['Pp changes 5'].values
l_mark = list()
for i in tqdm(range(len(df))):
    if price_chan[i] >= 0.01:
        l_mark.append(1)
    else:
        l_mark.append(0)
df['Markers 5'] = l_mark

price_chan = df['Pp changes 10'].values
l_mark = list()
for i in tqdm(range(len(df))):
    if price_chan[i] >= 0.01:
        l_mark.append(1)
    else:
        l_mark.append(0)
df['Markers 10'] = l_mark

price_chan = df['Pp changes 25'].values
l_mark = list()
for i in tqdm(range(len(df))):
    if price_chan[i] >= 0.01:
        l_mark.append(1)
    else:
        l_mark.append(0)
df['Markers 25'] = l_mark

price_chan = df['Pp changes 50'].values
l_mark = list()
for i in tqdm(range(len(df))):
    if price_chan[i] >= 0.01:
        l_mark.append(1)
    else:
        l_mark.append(0)
df['Markers 50'] = l_mark

price_chan = df['Pp changes 100'].values
l_mark = list()
for i in tqdm(range(len(df))):
    if price_chan[i] >= 0.01:
        l_mark.append(1)
    else:
        l_mark.append(0)
df['Markers 100'] = l_mark

100%|██████████| 25844702/25844702 [00:20<00:00, 1283988.17it/s]


In [32]:
df.head()

,datetimes,Mid Price,Imb_1_LvL,Imb_1_LvL_W,Log Imb1LvL,Imb_3_LvL,Imb_3_LvL_W,Log Imb3LvL,Sum Bid Volume_3,Sum Ask Volume_3,...,Pp changes 10,Pp changes 25,Pp changes 50,Pp changes 100,Markers 1,Markers 5,Markers 10,Markers 25,Markers 50,Markers 100
0,2020-07-29 12:25:06.554033,11108.25,2594633,0.980946,4.644061,2782721,0.967003,4.087857,2830198,47477,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1,2020-07-29 12:25:06.554647,11108.25,2594633,0.980946,4.644061,2782721,0.967003,4.087857,2830198,47477,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
2,2020-07-29 12:25:06.555250,11108.25,2594633,0.980946,4.644061,2782721,0.967003,4.087857,2830198,47477,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
3,2020-07-29 12:25:06.555849,11108.25,2594633,0.980946,4.644061,2782721,0.967003,4.087857,2830198,47477,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
4,2020-07-29 12:25:06.556386,11108.25,2594633,0.980946,4.644061,2782721,0.967003,4.087857,2830198,47477,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0


In [2]:
%%time
df_classificator = pd.read_csv('/home/brainiac/Vlad/Git Files/CryptoLp DataScience/Projects/05. Imbalanced/File for experiments.csv')

CPU times: user 49.3 s, sys: 6.9 s, total: 56.2 s
Wall time: 1min 5s


In [6]:
df_classificator.columns

Index(['datetimes', 'Mid Price', 'Imb_1_LvL', 'Imb_1_LvL_W', 'Log Imb1LvL',
       'Imb_3_LvL', 'Imb_3_LvL_W', 'Log Imb3LvL', 'Sum Bid Volume_3',
       'Sum Ask Volume_3', 'Imb_5_LvL', 'Imb_5_LvL_W', 'Log Imb5LvL',
       'Sum Bid Volume_5', 'Sum Ask Volume_5', 'Imb_10_LvL', 'Imb_10_LvL_W',
       'Log Imb10LvL', 'Sum Bid Volume_10', 'Sum Ask Volume_10',
       'Price Change 1', 'Price Change 5', 'Price Change 10',
       'Price Change 25', 'Price Change 50', 'Price Change 100',
       'Pp changes 1', 'Pp changes 5', 'Pp changes 10', 'Pp changes 25',
       'Pp changes 50', 'Pp changes 100', 'Markers 1', 'Markers 5',
       'Markers 10', 'Markers 25', 'Markers 50', 'Markers 100'],
      dtype='object')

In [7]:
X = df_classificator[['Imb_1_LvL','Imb_1_LvL_W','Log Imb1LvL',
                      'Imb_3_LvL','Imb_3_LvL_W','Log Imb3LvL','Sum Bid Volume_3','Sum Ask Volume_3',
                      'Imb_5_LvL','Imb_5_LvL_W','Log Imb5LvL','Sum Bid Volume_5','Sum Ask Volume_5',
                      'Imb_10_LvL','Imb_10_LvL_W','Log Imb10LvL','Sum Bid Volume_10','Sum Ask Volume_10',
                     ]]
y = df_classificator['Markers 1']

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=0)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test.values, pred, average='macro')

# Блок 2 Проверка баров

In [7]:
df = df[['datetimes','Mid Price',
         'bid_0_size','bid_1_size','bid_2_size','bid_3_size','bid_4_size','bid_5_size','bid_6_size','bid_7_size','bid_8_size','bid_9_size',
         'ask_0_size','ask_1_size','ask_2_size','ask_3_size','ask_4_size','ask_5_size','ask_6_size','ask_7_size','ask_8_size','ask_9_size',
        ]]

In [8]:
df.head()

,datetimes,Mid Price,bid_0_size,bid_1_size,bid_2_size,bid_3_size,bid_4_size,bid_5_size,bid_6_size,bid_7_size,...,ask_0_size,ask_1_size,ask_2_size,ask_3_size,ask_4_size,ask_5_size,ask_6_size,ask_7_size,ask_8_size,ask_9_size
0,2020-07-29 12:25:06.423343+00:00,11108.25,2470752,183215,27029,13019,123951,124582,129294,46695,...,61476,22340,129,289,1076,10150,10367,43269,10543,64703
1,2020-07-29 12:25:06.488583,11108.25,2470752,183215,27029,13019,123951,124582,129294,46695,...,61476,22340,129,289,1076,10150,10367,43269,10543,64703
2,2020-07-29 12:25:06.489924,11108.25,2470752,183215,27029,13019,123951,124582,129294,46695,...,61476,22340,129,289,1076,10150,10367,43269,10543,64703
3,2020-07-29 12:25:06.490491,11108.25,2470752,183215,27029,13019,123951,124582,129294,46695,...,61476,22340,129,289,4676,10150,10367,43269,10543,64703
4,2020-07-29 12:25:06.491082,11108.25,2472252,183215,27029,13019,123951,124582,129294,46695,...,61476,22340,129,289,4676,10150,10367,43269,10543,64703


In [9]:
for i in tqdm(range(len(df))):
    df['datetimes'].values[i] = pd.to_datetime(df['datetimes'].values[i]).strftime("%Y-%m-%d %H:%M:%S")

100%|██████████| 25844802/25844802 [35:36<00:00, 12095.65it/s]


In [10]:
%%time
df.index = df['datetimes'].values
df['datetimes'] = pd.to_datetime(df['datetimes'])

CPU times: user 4.35 s, sys: 7.96 ms, total: 4.36 s
Wall time: 4.35 s


In [11]:
df.head()

,datetimes,Mid Price,bid_0_size,bid_1_size,bid_2_size,bid_3_size,bid_4_size,bid_5_size,bid_6_size,bid_7_size,...,ask_0_size,ask_1_size,ask_2_size,ask_3_size,ask_4_size,ask_5_size,ask_6_size,ask_7_size,ask_8_size,ask_9_size
2020-07-29 12:25:06,2020-07-29 12:25:06,11108.25,2470752,183215,27029,13019,123951,124582,129294,46695,...,61476,22340,129,289,1076,10150,10367,43269,10543,64703
2020-07-29 12:25:06,2020-07-29 12:25:06,11108.25,2470752,183215,27029,13019,123951,124582,129294,46695,...,61476,22340,129,289,1076,10150,10367,43269,10543,64703
2020-07-29 12:25:06,2020-07-29 12:25:06,11108.25,2470752,183215,27029,13019,123951,124582,129294,46695,...,61476,22340,129,289,1076,10150,10367,43269,10543,64703
2020-07-29 12:25:06,2020-07-29 12:25:06,11108.25,2470752,183215,27029,13019,123951,124582,129294,46695,...,61476,22340,129,289,4676,10150,10367,43269,10543,64703
2020-07-29 12:25:06,2020-07-29 12:25:06,11108.25,2472252,183215,27029,13019,123951,124582,129294,46695,...,61476,22340,129,289,4676,10150,10367,43269,10543,64703


In [13]:
Bt_datetime = df.groupby(pd.Grouper(key='datetimes', freq='1s')).max().index
Mean_price = df.groupby(pd.Grouper(key='datetimes', freq='1s')).mean()['Mid Price'].values

Bid_0_Volume = df.groupby(pd.Grouper(key='datetimes', freq='1s')).mean()['bid_0_size'].values
Bid_1_Volume = df.groupby(pd.Grouper(key='datetimes', freq='1s')).mean()['bid_1_size'].values
Bid_2_Volume = df.groupby(pd.Grouper(key='datetimes', freq='1s')).mean()['bid_2_size'].values
Bid_3_Volume = df.groupby(pd.Grouper(key='datetimes', freq='1s')).mean()['bid_3_size'].values
Bid_4_Volume = df.groupby(pd.Grouper(key='datetimes', freq='1s')).mean()['bid_4_size'].values
Bid_5_Volume = df.groupby(pd.Grouper(key='datetimes', freq='1s')).mean()['bid_5_size'].values
Bid_6_Volume = df.groupby(pd.Grouper(key='datetimes', freq='1s')).mean()['bid_6_size'].values
Bid_7_Volume = df.groupby(pd.Grouper(key='datetimes', freq='1s')).mean()['bid_7_size'].values
Bid_8_Volume = df.groupby(pd.Grouper(key='datetimes', freq='1s')).mean()['bid_8_size'].values
Bid_9_Volume = df.groupby(pd.Grouper(key='datetimes', freq='1s')).mean()['bid_9_size'].values

Ask_0_Volume = df.groupby(pd.Grouper(key='datetimes', freq='1s')).mean()['ask_0_size'].values
Ask_1_Volume = df.groupby(pd.Grouper(key='datetimes', freq='1s')).mean()['ask_1_size'].values
Ask_2_Volume = df.groupby(pd.Grouper(key='datetimes', freq='1s')).mean()['ask_2_size'].values
Ask_3_Volume = df.groupby(pd.Grouper(key='datetimes', freq='1s')).mean()['ask_3_size'].values
Ask_4_Volume = df.groupby(pd.Grouper(key='datetimes', freq='1s')).mean()['ask_4_size'].values
Ask_5_Volume = df.groupby(pd.Grouper(key='datetimes', freq='1s')).mean()['ask_5_size'].values
Ask_6_Volume = df.groupby(pd.Grouper(key='datetimes', freq='1s')).mean()['ask_6_size'].values
Ask_7_Volume = df.groupby(pd.Grouper(key='datetimes', freq='1s')).mean()['ask_7_size'].values
Ask_8_Volume = df.groupby(pd.Grouper(key='datetimes', freq='1s')).mean()['ask_8_size'].values
Ask_9_Volume = df.groupby(pd.Grouper(key='datetimes', freq='1s')).mean()['ask_9_size'].values

In [14]:
df_Bt_bars = pd.DataFrame(columns=['datetime',
                                   'Mean price',])
df_Bt_bars['datetime'] = Bt_datetime
df_Bt_bars['Mean price'] = Mean_price

df_Bt_bars['Bid Size 0'] = Bid_0_Volume
df_Bt_bars['Bid Size 1'] = Bid_1_Volume
df_Bt_bars['Bid Size 2'] = Bid_2_Volume
df_Bt_bars['Bid Size 3'] = Bid_3_Volume
df_Bt_bars['Bid Size 4'] = Bid_4_Volume
df_Bt_bars['Bid Size 5'] = Bid_5_Volume
df_Bt_bars['Bid Size 6'] = Bid_6_Volume
df_Bt_bars['Bid Size 7'] = Bid_7_Volume
df_Bt_bars['Bid Size 8'] = Bid_8_Volume
df_Bt_bars['Bid Size 9'] = Bid_9_Volume

df_Bt_bars['Ask Size 0'] = Ask_0_Volume
df_Bt_bars['Ask Size 1'] = Ask_1_Volume
df_Bt_bars['Ask Size 2'] = Ask_2_Volume
df_Bt_bars['Ask Size 3'] = Ask_3_Volume
df_Bt_bars['Ask Size 4'] = Ask_4_Volume
df_Bt_bars['Ask Size 5'] = Ask_5_Volume
df_Bt_bars['Ask Size 6'] = Ask_6_Volume
df_Bt_bars['Ask Size 7'] = Ask_7_Volume
df_Bt_bars['Ask Size 8'] = Ask_8_Volume
df_Bt_bars['Ask Size 9'] = Ask_9_Volume

In [15]:
df_Bt_bars.head()

,datetime,Mean price,Bid Size 0,Bid Size 1,Bid Size 2,Bid Size 3,Bid Size 4,Bid Size 5,Bid Size 6,Bid Size 7,...,Ask Size 0,Ask Size 1,Ask Size 2,Ask Size 3,Ask Size 4,Ask Size 5,Ask Size 6,Ask Size 7,Ask Size 8,Ask Size 9
0,2020-07-29 12:25:06,11108.250000,2.576580e+06,1.857827e+05,26943.285714,12768.480519,128235.606061,124881.610390,129172.086580,41761.601732,...,26141.259740,22227.549784,129.000000,565.069264,6572.160173,10162.043290,23011.298701,26656.922078,18490.610390,54537.982684
1,2020-07-29 12:25:07,11108.250000,2.642730e+06,1.833470e+05,26729.000000,12415.000000,124073.000000,136117.000000,128682.000000,40828.000000,...,25199.000000,22149.000000,129.000000,885.000000,10439.000000,10176.000000,37713.000000,10046.395349,20321.604651,10710.000000
2,2020-07-29 12:25:08,11108.612989,2.010524e+06,1.065650e+06,139961.822064,24205.896797,69869.683274,127416.174377,134830.416370,96912.697509,...,22029.412811,6084.587189,680.530249,9148.950178,10311.188612,30184.345196,17495.035587,21929.911032,14498.000000,15667.220641
3,2020-07-29 12:25:09,11108.750000,3.110557e+06,3.262952e+05,78025.474138,41191.758621,76764.982759,153386.275862,134320.362069,93004.000000,...,12619.551724,129.000000,885.000000,956.241379,7158.758621,34583.758621,6997.758621,21268.517241,7264.758621,11363.000000
4,2020-07-29 12:25:10,11111.653409,2.275050e+06,8.704634e+04,74667.656566,201104.075758,90906.712121,440745.214646,581021.053030,341354.118687,...,159681.686869,615.037879,8709.550505,397.553030,251.421717,68783.093434,63643.184343,61361.608586,26511.515152,55290.179293


In [18]:
df.shape
df_Bt_bars.shape

(25844802, 22)

(978455, 22)

# Блок 3 Разные имбалансы

In [22]:
df_Bt_bars['Sum Bid Volume'] = df_Bt_bars['Bid Size 0']+df_Bt_bars['Bid Size 1']+df_Bt_bars['Bid Size 2']+df_Bt_bars['Bid Size 3']+df_Bt_bars['Bid Size 4']+\
                               df_Bt_bars['Bid Size 5']+df_Bt_bars['Bid Size 6']+df_Bt_bars['Bid Size 7']+df_Bt_bars['Bid Size 8']+df_Bt_bars['Bid Size 9']

df_Bt_bars['Sum Ask Volume'] = df_Bt_bars['Ask Size 0']+df_Bt_bars['Ask Size 1']+df_Bt_bars['Ask Size 2']+df_Bt_bars['Ask Size 3']+df_Bt_bars['Ask Size 4']+\
                               df_Bt_bars['Ask Size 5']+df_Bt_bars['Ask Size 6']+df_Bt_bars['Ask Size 7']+df_Bt_bars['Ask Size 8']+df_Bt_bars['Ask Size 9']

In [23]:
df_Bt_bars.head(3)

,datetime,Mean price,Bid Size 0,Bid Size 1,Bid Size 2,Bid Size 3,Bid Size 4,Bid Size 5,Bid Size 6,Bid Size 7,...,Ask Size 2,Ask Size 3,Ask Size 4,Ask Size 5,Ask Size 6,Ask Size 7,Ask Size 8,Ask Size 9,Sum Bid Volume,Sum Ask Volume
0,2020-07-29 12:25:06,11108.250000,2.576580e+06,1.857827e+05,26943.285714,12768.480519,128235.606061,124881.610390,129172.08658,41761.601732,...,129.000000,565.069264,6572.160173,10162.043290,23011.298701,26656.922078,18490.610390,54537.982684,3.367132e+06,188493.896104
1,2020-07-29 12:25:07,11108.250000,2.642730e+06,1.833470e+05,26729.000000,12415.000000,124073.000000,136117.000000,128682.00000,40828.000000,...,129.000000,885.000000,10439.000000,10176.000000,37713.000000,10046.395349,20321.604651,10710.000000,3.453435e+06,147768.000000
2,2020-07-29 12:25:08,11108.612989,2.010524e+06,1.065650e+06,139961.822064,24205.896797,69869.683274,127416.174377,134830.41637,96912.697509,...,680.530249,9148.950178,10311.188612,30184.345196,17495.035587,21929.911032,14498.000000,15667.220641,3.834428e+06,148029.181495


In [30]:
%%time
# Имбалансы первого уровня
df_Bt_bars['Imb_1_LvL'] = (df_Bt_bars['Bid Size 0']-df_Bt_bars['Ask Size 0'])
df_Bt_bars['Imb_1_LvL_W'] = (df_Bt_bars['Bid Size 0']-df_Bt_bars['Ask Size 0'])/(df_Bt_bars['Bid Size 0']+df_Bt_bars['Ask Size 0'])
df_Bt_bars['Log Imb1LvL'] = np.log(df_Bt_bars['Bid Size 0'])-np.log(df_Bt_bars['Ask Size 0'])
#***********************

# Имбалансы трех уровней
df_Bt_bars['Sum Bid Volume_3'] = df_Bt_bars['Bid Size 0']+df_Bt_bars['Bid Size 1']+df_Bt_bars['Bid Size 2']
df_Bt_bars['Sum Ask Volume_3'] = df_Bt_bars['Ask Size 0']+df_Bt_bars['Ask Size 1']+df_Bt_bars['Ask Size 2']

df_Bt_bars['Imb_3_LvL'] = (df_Bt_bars['Sum Bid Volume_3']-df_Bt_bars['Sum Ask Volume_3'])
df_Bt_bars['Imb_3_LvL_W'] = (df_Bt_bars['Sum Bid Volume_3']-df_Bt_bars['Sum Ask Volume_3'])/(df_Bt_bars['Sum Bid Volume_3']+df_Bt_bars['Sum Ask Volume_3'])
df_Bt_bars['Log Imb3LvL'] = np.log(df_Bt_bars['Sum Bid Volume_3'])-np.log(df_Bt_bars['Sum Ask Volume_3'])
#***********************

# Имбалансы пяти уровней
df_Bt_bars['Sum Bid Volume_5'] = df_Bt_bars['Bid Size 0']+df_Bt_bars['Bid Size 1']+df_Bt_bars['Bid Size 2']+df_Bt_bars['Bid Size 3']+df_Bt_bars['Bid Size 4']
df_Bt_bars['Sum Ask Volume_5'] = df_Bt_bars['Ask Size 0']+df_Bt_bars['Ask Size 1']+df_Bt_bars['Ask Size 2']+df_Bt_bars['Ask Size 3']+df_Bt_bars['Ask Size 4']

df_Bt_bars['Imb_5_LvL'] = (df_Bt_bars['Sum Bid Volume_5']-df_Bt_bars['Sum Ask Volume_5'])
df_Bt_bars['Imb_5_LvL_W'] = (df_Bt_bars['Sum Bid Volume_5']-df_Bt_bars['Sum Ask Volume_5'])/(df_Bt_bars['Sum Bid Volume_5']+df_Bt_bars['Sum Ask Volume_5'])
df_Bt_bars['Log Imb5LvL'] = np.log(df_Bt_bars['Sum Bid Volume_5'])-np.log(df_Bt_bars['Sum Ask Volume_5'])
#***********************

# Имбалансы десяти уровней
df_Bt_bars['Sum Bid Volume_10'] = df_Bt_bars['Bid Size 0']+df_Bt_bars['Bid Size 1']+df_Bt_bars['Bid Size 2']+df_Bt_bars['Bid Size 3']+df_Bt_bars['Bid Size 4']+\
                                  df_Bt_bars['Bid Size 5']+df_Bt_bars['Bid Size 6']+df_Bt_bars['Bid Size 7']+df_Bt_bars['Bid Size 8']+df_Bt_bars['Bid Size 9']

df_Bt_bars['Sum Ask Volume_10'] = df_Bt_bars['Ask Size 0']+df_Bt_bars['Ask Size 1']+df_Bt_bars['Ask Size 2']+df_Bt_bars['Ask Size 3']+df_Bt_bars['Ask Size 4']+\
                                  df_Bt_bars['Ask Size 5']+df_Bt_bars['Ask Size 6']+df_Bt_bars['Ask Size 7']+df_Bt_bars['Ask Size 8']+df_Bt_bars['Ask Size 9']

df_Bt_bars['Imb_10_LvL'] = (df_Bt_bars['Sum Bid Volume_10']-df_Bt_bars['Sum Ask Volume_10'])
df_Bt_bars['Imb_10_LvL_W'] = (df_Bt_bars['Sum Bid Volume_10']-df_Bt_bars['Sum Ask Volume_10'])/(df_Bt_bars['Sum Bid Volume_10']+df_Bt_bars['Sum Ask Volume_10'])
df_Bt_bars['Log Imb10LvL'] = np.log(df_Bt_bars['Sum Bid Volume_10'])-np.log(df_Bt_bars['Sum Ask Volume_10'])
#***********************

CPU times: user 462 ms, sys: 8.74 ms, total: 470 ms
Wall time: 173 ms


In [32]:
df_Bt_bars['Price changes'] = df_Bt_bars['Mean price'].diff()

In [33]:
mid_price = df_Bt_bars['Mean price'].values
Price_changes_percent = [0]
for i in tqdm(range(1, len(df_Bt_bars))):
    a = ((mid_price[i]-mid_price[i-1])/mid_price[i-1])*100
    Price_changes_percent.append(a)
df_Bt_bars['Price changes percent'] = Price_changes_percent

100%|██████████| 978454/978454 [00:01<00:00, 550393.03it/s]


In [35]:
df_Bt_bars = df_Bt_bars.drop(['Bid Size 0','Bid Size 1','Bid Size 2','Bid Size 3','Bid Size 4','Bid Size 5','Bid Size 6','Bid Size 7','Bid Size 8','Bid Size 9',
                              'Ask Size 0','Ask Size 1','Ask Size 2','Ask Size 3','Ask Size 4','Ask Size 5','Ask Size 6','Ask Size 7','Ask Size 8','Ask Size 9',
                             ],axis=1)

In [49]:
df_Bt_bars.head()

,datetime,Mean price,Imb_1_LvL,Imb_1_LvL_W,Log Imb1LvL,Imb_3_LvL,Imb_3_LvL_W,Log Imb3LvL,Imb_5_LvL,Imb_5_LvL_W,Log Imb5LvL,Imb_10_LvL,Imb_10_LvL_W,Log Imb10LvL,Price changes,Price changes percent
0,2020-07-29 12:25:06,11108.250000,2.550439e+06,0.979912,4.590703,2.740808e+06,0.965820,4.052029,2.874675e+06,0.962735,3.964050,3.178639e+06,0.893974,2.882751,NaN,0.000000
1,2020-07-29 12:25:07,11108.250000,2.617531e+06,0.981110,4.652764,2.805329e+06,0.967260,4.095813,2.930493e+06,0.961418,3.928634,3.305667e+06,0.917934,3.151481,0.000000,0.000000
2,2020-07-29 12:25:08,11108.612989,1.988495e+06,0.978323,4.513772,3.187341e+06,0.982253,4.715750,3.261957e+06,0.971264,4.228275,3.686398e+06,0.925659,3.254366,0.362989,0.003268
3,2020-07-29 12:25:09,11108.750000,3.097937e+06,0.991919,5.507310,3.501244e+06,0.992272,5.552226,3.611086e+06,0.988098,5.118221,4.078006e+06,0.951814,3.701437,0.137011,0.001233
4,2020-07-29 12:25:10,11111.653409,2.115368e+06,0.868830,2.656575,2.267758e+06,0.870283,2.668490,2.559119e+06,0.882933,2.777839,3.780014e+06,0.809338,2.250212,2.903409,0.026136


In [48]:
df_Bt_bars.columns

Index(['datetime', 'Mean price', 'Imb_1_LvL', 'Imb_1_LvL_W', 'Log Imb1LvL',
       'Imb_3_LvL', 'Imb_3_LvL_W', 'Log Imb3LvL', 'Imb_5_LvL', 'Imb_5_LvL_W',
       'Log Imb5LvL', 'Imb_10_LvL', 'Imb_10_LvL_W', 'Log Imb10LvL',
       'Price changes', 'Price changes percent'],
      dtype='object')

In [124]:
df_Bt_bars['Imb_5_LvL_W'].describe()

count    344305.000000
mean         -0.070350
std           0.615344
min          -0.998600
25%          -0.633692
50%          -0.154539
75%           0.517599
max           0.998756
Name: Imb_5_LvL_W, dtype: float64

In [125]:
k = 0.97
len(df_Bt_bars['Imb_5_LvL_W'])
len(df_Bt_bars[df_Bt_bars['Imb_5_LvL_W']>k])
len(df_Bt_bars[df_Bt_bars['Imb_5_LvL_W']<-k])

978455

2609

2989

In [126]:
df_imbalance_OB = pd.concat([df_Bt_bars[df_Bt_bars['Imb_5_LvL_W']>k], df_Bt_bars[df_Bt_bars['Imb_5_LvL_W']<-k]])

In [127]:
df_imbalance_OB.head()

,datetime,Mean price,Imb_1_LvL,Imb_1_LvL_W,Log Imb1LvL,Imb_3_LvL,Imb_3_LvL_W,Log Imb3LvL,Imb_5_LvL,Imb_5_LvL_W,Log Imb5LvL,Imb_10_LvL,Imb_10_LvL_W,Log Imb10LvL,Price changes,Price changes percent
2,2020-07-29 12:25:08,11108.612989,1.988495e+06,0.978323,4.513772,3.187341e+06,0.982253,4.715750,3.261957e+06,0.971264,4.228275,3.686398e+06,0.925659,3.254366,0.362989,0.003268
3,2020-07-29 12:25:09,11108.750000,3.097937e+06,0.991919,5.507310,3.501244e+06,0.992272,5.552226,3.611086e+06,0.988098,5.118221,4.078006e+06,0.951814,3.701437,0.137011,0.001233
258,2020-07-29 12:29:24,11087.555233,5.570187e+05,0.985815,4.941612,2.610236e+06,0.994136,5.829134,4.827639e+06,0.988266,5.132488,6.157162e+06,0.940974,3.492961,1.637451,0.014771
351,2020-07-29 12:30:57,11097.750000,4.642400e+06,0.990832,5.380575,4.949010e+06,0.990230,5.316708,5.033998e+06,0.975731,4.399495,5.485392e+06,0.938011,3.442467,0.000000,0.000000
352,2020-07-29 12:30:58,11097.750000,4.595338e+06,0.990761,5.372805,4.945172e+06,0.990243,5.318023,5.030343e+06,0.975768,4.401028,5.480881e+06,0.936446,3.416727,0.000000,0.000000


In [128]:
# df_Bt_bars[(df_Bt_bars['datetime']>=str(pd.to_datetime('2020-07-29 12:25:09')-timedelta(seconds=1)))&
#            (df_Bt_bars['datetime']<str(pd.to_datetime('2020-07-29 12:25:09')+timedelta(minutes=1)))]

In [129]:
dict_PC = {}
for i in tqdm(range(len(df_imbalance_OB))):
    a = df_Bt_bars[(df_Bt_bars['datetime']>=str(pd.to_datetime(df_imbalance_OB['datetime'].values[i])-timedelta(seconds=1)))&
                   (df_Bt_bars['datetime']<str(pd.to_datetime(df_imbalance_OB['datetime'].values[i])+timedelta(minutes=1)))]
    if a['Price changes percent'].max() > 0.01:
        dict_PC.update({df_imbalance_OB['datetime'].iloc[i].strftime('%Y-%m-%d %H:%M:%S'): a})

100%|██████████| 5598/5598 [00:11<00:00, 507.82it/s]


In [130]:
len(dict_PC.keys())
len(df_imbalance_OB)

len(dict_PC.keys())/len(df_imbalance_OB)

3094

5598

0.5526973919256878

# Блок 4 Попытка примениения классификатора

In [172]:
df_Bt_bars.head(1)

,datetime,Mean price,Imb_1_LvL,Imb_1_LvL_W,Log Imb1LvL,Imb_3_LvL,Imb_3_LvL_W,Log Imb3LvL,Imb_5_LvL,Imb_5_LvL_W,Log Imb5LvL,Imb_10_LvL,Imb_10_LvL_W,Log Imb10LvL,Price changes,Price changes percent
0,2020-07-29 12:25:06,11108.25,2.550439e+06,0.979912,4.590703,2.740808e+06,0.96582,4.052029,2874675.0,0.962735,3.96405,3.178639e+06,0.893974,2.882751,NaN,0.0


In [175]:
df_classif = df_Bt_bars[['datetime','Mean price','Imb_1_LvL_W','Log Imb1LvL','Imb_3_LvL_W','Log Imb3LvL','Imb_5_LvL_W','Log Imb5LvL','Imb_10_LvL_W','Log Imb10LvL',]]
df_classif = df_classif.dropna()

In [176]:
df_classif.head()

,datetime,Mean price,Imb_1_LvL_W,Log Imb1LvL,Imb_3_LvL_W,Log Imb3LvL,Imb_5_LvL_W,Log Imb5LvL,Imb_10_LvL_W,Log Imb10LvL
0,2020-07-29 12:25:06,11108.250000,0.979912,4.590703,0.965820,4.052029,0.962735,3.964050,0.893974,2.882751
1,2020-07-29 12:25:07,11108.250000,0.981110,4.652764,0.967260,4.095813,0.961418,3.928634,0.917934,3.151481
2,2020-07-29 12:25:08,11108.612989,0.978323,4.513772,0.982253,4.715750,0.971264,4.228275,0.925659,3.254366
3,2020-07-29 12:25:09,11108.750000,0.991919,5.507310,0.992272,5.552226,0.988098,5.118221,0.951814,3.701437
4,2020-07-29 12:25:10,11111.653409,0.868830,2.656575,0.870283,2.668490,0.882933,2.777839,0.809338,2.250212


In [145]:
price_chan = df_classif['Price changes percent'].values
l_mark = list()
for i in tqdm(range(len(df_classif))):
    if price_chan[i] >= 0.01:
        l_mark.append(1)
    else:
        l_mark.append(0)
df_classif['Marker'] = l_mark

100%|██████████| 343925/343925 [00:00<00:00, 1057633.97it/s]


In [146]:
df_classif.head(2)

,datetime,Imb_1_LvL_W,Log Imb1LvL,Imb_3_LvL_W,Log Imb3LvL,Imb_5_LvL_W,Log Imb5LvL,Imb_10_LvL_W,Log Imb10LvL,Price changes percent,Marker
0,2020-07-29 12:25:06,0.979912,4.590703,0.96582,4.052029,0.962735,3.964050,0.893974,2.882751,0.0,0
1,2020-07-29 12:25:07,0.981110,4.652764,0.96726,4.095813,0.961418,3.928634,0.917934,3.151481,0.0,0


In [165]:
df_classif['Marker'].value_counts(normalize=True)

0    0.986706
1    0.013294
Name: Marker, dtype: float64

In [147]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X, y = df_classif.drop(['datetime','Price changes percent','Marker'],axis=1), df_classif[['Marker']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [148]:
clf = RandomForestClassifier(random_state=0)

In [150]:
%%time
clf.fit(X_train, y_train)

/home/brainiac/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



CPU times: user 1min 10s, sys: 54.3 ms, total: 1min 10s
Wall time: 1min 10s


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [151]:
pred = clf.predict(X_test)

In [169]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test.values, pred)

from sklearn.metrics import f1_score
f1_score(y_test.values, pred, average='macro')

0.9874624921495196

0.6101829051700984

In [171]:
from sklearn.metrics import classification_report
report = classification_report(y_test.values, pred)
print(report)

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     84810
           1       0.71      0.13      0.23      1172

    accuracy                           0.99     85982
   macro avg       0.85      0.57      0.61     85982
weighted avg       0.98      0.99      0.98     85982



In [156]:
len(y_test.values)
len(pred)

85982

85982

In [158]:
df_vlad = X_test

In [160]:
df_vlad['Predicts'] = y_test

In [161]:
df_vlad.head()

,Imb_1_LvL_W,Log Imb1LvL,Imb_3_LvL_W,Log Imb3LvL,Imb_5_LvL_W,Log Imb5LvL,Imb_10_LvL_W,Log Imb10LvL,Predicts
27379,-0.319383,-0.661919,-0.333105,-0.692634,-0.349062,-0.728750,-0.400683,-0.848923,0
836188,-0.085462,-0.171342,-0.134660,-0.270965,-0.122068,-0.245360,-0.105099,-0.210976,0
164164,-0.858552,-2.575623,-0.870338,-2.668940,-0.890911,-2.852650,-0.758841,-1.986953,0
92970,0.720797,1.818604,0.653837,1.563944,0.358817,0.751056,0.361195,0.756520,0
90414,0.096761,0.194129,0.148200,0.298598,0.173607,0.350767,0.117049,0.235177,0


In [162]:
df_vlad[df_vlad['Predicts']==1]

,Imb_1_LvL_W,Log Imb1LvL,Imb_3_LvL_W,Log Imb3LvL,Imb_5_LvL_W,Log Imb5LvL,Imb_10_LvL_W,Log Imb10LvL,Predicts
53928,0.655705,1.570484,0.605217,1.402679,0.755614,1.971825,0.828667,2.367736,1
34705,0.900237,2.946941,0.895969,2.902800,0.858934,2.578529,0.733395,1.872067,1
877461,0.455281,0.982685,-0.018407,-0.036819,-0.242181,-0.494179,-0.059880,-0.119904,1
42077,-0.816883,-2.294755,-0.540581,-1.209952,-0.223491,-0.454655,0.345347,0.720301,1
887241,0.919748,3.174778,0.934345,3.383114,0.930640,3.326298,0.895334,2.896373,1
...,...,...,...,...,...,...,...,...,...
874950,0.904609,2.994044,0.881888,2.768395,0.811772,2.264409,0.464649,1.006448,1
23708,0.720144,1.815889,0.732400,1.867769,0.725644,1.838927,0.819157,2.308498,1
52029,0.419863,0.895052,0.459671,0.993788,0.675293,1.640821,0.776523,2.073106,1
114599,0.991142,5.415151,0.973317,4.303428,0.985041,4.888098,0.946744,3.598809,1


In [ ]:
df_Bt_bars[(df_Bt_bars['datetime']<str(pd.to_datetime(df_imbalance_OB['datetime'].values[i])+
                                                 timedelta(minutes=1)))]

In [41]:
df_imbalance_OB = df_imbalance_OB.sort_values(by=['datetime'])
df_imbalance_OB.index = range(len(df_imbalance_OB))

In [42]:
df_imbalance_OB.head(1)

,datetime,Mean price,Bid Volume,Ask Volume,Imbalance,Log Imb,Price changes,Price changes percent
0,2020-07-29 12:49:26,11105.40982,6.419615e+06,106101.853707,0.967482,4.102714,1.612945,0.014526


In [44]:
df_imbalance_OB.shape

(919, 8)

In [50]:
df_imbalance_OB.head()

,datetime,Mean price,Bid Volume,Ask Volume,Imbalance,Log Imb,Price changes,Price changes percent
0,2020-07-29 12:49:26,11105.409820,6.419615e+06,1.061019e+05,0.967482,4.102714,1.612945,0.014526
1,2020-07-29 12:51:12,11118.996109,5.182115e+06,9.226887e+04,0.965012,4.028262,0.246109,0.002213
2,2020-07-29 12:51:13,11120.229927,6.710034e+06,1.012955e+05,0.970257,4.193318,1.233818,0.011096
3,2020-07-29 12:55:31,11143.685552,5.783177e+06,1.042307e+05,0.964592,4.016102,0.435552,0.003909
4,2020-07-29 12:57:49,11138.750000,4.288170e+04,3.220769e+06,-0.973722,-4.318930,0.000000,0.000000


In [1]:
df_Bt_bars[df_Bt_bars['datetime'] > '2020-07-29 12:51:12'].head(1)

NameError: name 'df_Bt_bars' is not defined

In [36]:
dict_PC = {}
for i in tqdm(range(len(df_imbalance_OB))):
    a = df_Bt_bars[(df_Bt_bars['datetime']>=str(pd.to_datetime(df_imbalance_OB['datetime'].values[i])-
                                                  timedelta(seconds=10)))&
                    (df_Bt_bars['datetime']<str(pd.to_datetime(df_imbalance_OB['datetime'].values[i])+
                                                 timedelta(minutes=1)))]
    if len(a) != 0:
        dict_PC.update({df_imbalance_OB['datetime'].iloc[i].strftime('%Y-%m-%d %H:%M:%S'): a})

100%|██████████| 919/919 [01:16<00:00, 12.00it/s] 


In [26]:
df_Bt_bars.shape

(978455, 8)

In [37]:
list_date = list(dict_PC.keys())

In [38]:
len(list_date)
len(df_imbalance_OB['datetime'].values)

919

919

### Окончание проверки баров

In [6]:
# %%time
# df_test['datetimes'] = pd.to_datetime(df_test['datetimes'])

In [7]:
df = df[['datetimes','Sum Bid Volume','Sum Ask Volume','Mid Price']]

In [8]:
df.head()

,datetimes,Sum Bid Volume,Sum Ask Volume,Mid Price
0,2020-07-29 12:25:06.423343+00:00,3264111,224342,11108.25
1,2020-07-29 12:25:06.488583,3264111,224342,11108.25
2,2020-07-29 12:25:06.489924,3264111,224342,11108.25
3,2020-07-29 12:25:06.490491,3264111,227942,11108.25
4,2020-07-29 12:25:06.491082,3265611,227942,11108.25


In [9]:
df['Imbalance'] = (df['Sum Bid Volume']-df['Sum Ask Volume'])/(df['Sum Bid Volume']+df['Sum Ask Volume'])
df['Log Imb'] = np.log(df['Sum Bid Volume'])-np.log(df['Sum Ask Volume'])
df['Price changes'] = df['Mid Price'].diff()

In [10]:
mid_price = df['Mid Price'].values
Price_changes_percent = [0]
for i in tqdm(range(1, len(df))):
    a = ((mid_price[i]-mid_price[i-1])/mid_price[i-1])*100
    Price_changes_percent.append(a)
df['Price changes percent'] = Price_changes_percent

100%|██████████| 25844801/25844801 [00:38<00:00, 674474.74it/s]


In [11]:
df.head(3)

,datetimes,Sum Bid Volume,Sum Ask Volume,Mid Price,Imbalance,Log Imb,Price changes,Price changes percent
0,2020-07-29 12:25:06.423343+00:00,3264111,224342,11108.25,0.87138,2.677571,NaN,0.0
1,2020-07-29 12:25:06.488583,3264111,224342,11108.25,0.87138,2.677571,0.0,0.0
2,2020-07-29 12:25:06.489924,3264111,224342,11108.25,0.87138,2.677571,0.0,0.0


In [12]:
df['Log Imb'].describe()

count    2.584480e+07
mean    -1.194718e-01
std      1.766877e+00
min     -6.645408e+00
25%     -1.456671e+00
50%     -1.942187e-01
75%      1.236786e+00
max      6.943186e+00
Name: Log Imb, dtype: float64

In [13]:
df_plus_Imbalance = df[df['Log Imb']>5.0]

In [14]:
df_plus_Imbalance.head(25)

,datetimes,Sum Bid Volume,Sum Ask Volume,Mid Price,Imbalance,Log Imb,Price changes,Price changes percent
266804,2020-07-29 13:05:31.642532900,4666062,25323,11123.00,0.989204,5.216358,0.25,0.002248
266805,2020-07-29 13:05:31.649815300,4514191,25323,11123.25,0.988843,5.183268,0.25,0.002248
266806,2020-07-29 13:05:31.649929800,4826091,25323,11123.25,0.989561,5.250079,0.00,0.000000
266908,2020-07-29 13:05:31.717366500,5392429,24938,11123.75,0.990793,5.376358,0.00,0.000000
266909,2020-07-29 13:05:31.717433,5432137,24938,11123.75,0.990860,5.383695,0.00,0.000000
266910,2020-07-29 13:05:31.717475400,5368205,24938,11123.75,0.990752,5.371856,0.00,0.000000
266911,2020-07-29 13:05:31.717518200,5432137,24938,11123.75,0.990860,5.383695,0.00,0.000000
266912,2020-07-29 13:05:31.717581700,5440166,24938,11123.75,0.990874,5.385172,0.00,0.000000
266913,2020-07-29 13:05:31.717835700,5440166,24938,11123.75,0.990874,5.385172,0.00,0.000000
266914,2020-07-29 13:05:31.717883,5445388,24938,11123.75,0.990882,5.386132,0.00,0.000000


In [101]:
marker_list = list()
for i in tqdm(range(len(df_test))):
    if (abs(df_test['Log Imb'].values[i]) >= 0) & (abs(df_test['Log Imb'].values[i]) < 2):
        marker_list.append('low')
    elif (abs(df_test['Log Imb'].values[i]) >= 2) & (abs(df_test['Log Imb'].values[i]) < 4):
        marker_list.append('medium')
    elif (abs(df_test['Log Imb'].values[i]) >= 4) & (abs(df_test['Log Imb'].values[i]) < 6):
        marker_list.append('high')
    else:
        marker_list.append('outlier')

100%|██████████| 25844802/25844802 [03:14<00:00, 132640.00it/s]


In [102]:
df_test['Markers'] = marker_list

In [103]:
df_test.head(3)

,datetimes,Mid Price,Imbalance,Log Imb,Markers
0,2020-07-29 12:25:06.423343+00:00,11108.25,0.87138,2.677571,medium
1,2020-07-29 12:25:06.488583,11108.25,0.87138,2.677571,medium
2,2020-07-29 12:25:06.489924,11108.25,0.87138,2.677571,medium


In [104]:
# Построение графиков
def show_plot_plotly(df_data:pd.DataFrame,
                     left_border,
                     right_border,
                     x_,
                     y_,
                     color_:str,
                     title_:str,             
             ):  
    fig = px.scatter(df_data[left_border:right_border], 
                 x=x_, 
                 y=y_, 
                 color=color_, 
                 #color_discrete_sequence=['blue','darkorange','green','red','gold', 'black'], 
                 title=title_
                )    
    return fig

In [107]:
df_test.shape

(25844802, 5)

In [108]:
df_result = df_test[:5_000_000]
fig = show_plot_plotly(df_data=df_result,
                       left_border = 0,
                       right_border=len(df_result),
                       x_='datetimes',
                       y_='Mid Price',
                       color_='Markers',
                       title_= 'Имбаланс объемов'
                      )
plotly.offline.plot(fig, filename='/home/brainiac/Vlad/Git Files/CryptoLp DataScience/DataSets/Orderbook/Имбаланс.html')

'/home/brainiac/Vlad/Git Files/CryptoLp DataScience/DataSets/Orderbook/Имбаланс.html'

In [ ]:
df_test